# AutoML Classification - Titantic 

This project aims to create a simple script to help identify the best machine learning algorithm for a classification problem.  The Kaggle Titanic project is used to run through every variable combination with 8 ML algorithms.  No hyperparameters are optimized here; after this project I will revisit each algorithm individually and optimize the hyperpameters then.

In [1]:
FileName = 'Titanic_Logistical_regression_1'

In [2]:
import pandas as pd
import pylab as pl
import numpy as np
import scipy.optimize as opt
from sklearn import preprocessing
%matplotlib inline 
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
import xgboost

C:\Users\peter\Anaconda3\lib\site-packages\dask\dataframe\utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
from sklearn import model_selection
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import CategoricalNB

In [4]:
train_df = pd.read_csv("train.csv")

##### Adjust Sex; Male = 0, Female = 1
Adjust Embark; C = 0, Q = 1, S = 2

Create Family member (sum of sibliing/spouse and parent/child)

In [5]:
train_df['Embarked'].replace(to_replace=['C','Q','S'], value=[0,1,2],inplace=True)
train_df['Sex'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
train_df['FamMeb']=train_df['SibSp']+train_df['Parch']
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamMeb
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,2.0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,0.0,1
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,2.0,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,2.0,1
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,2.0,0


Missing Age and Embarked values are replaced with the Median.  For embarked this isn't likely an issue as only 2 data points are missing, for Age this may or may not be best.  

In [6]:
train_df['Age'] = train_df['Age'].replace(np.nan, train_df['Age'].median())
train_df['Embarked'] = train_df['Embarked'].replace(np.nan, train_df['Embarked'].median())

In [7]:
train_df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'FamMeb'],
      dtype='object')

Name, Cabin, and Ticket are dropped as they have no trend to survivablity.

In [8]:
train_df = train_df.drop(['Name', 'Cabin', 'Ticket'], axis = 1)
train_df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamMeb
0,1,0,3,0,22.0,1,0,7.2500,2.0,1
1,2,1,1,1,38.0,1,0,71.2833,0.0,1
2,3,1,3,1,26.0,0,0,7.9250,2.0,0
3,4,1,1,1,35.0,1,0,53.1000,2.0,1
4,5,0,3,0,35.0,0,0,8.0500,2.0,0


In [9]:
a = train_df.columns.to_list()
a.remove('PassengerId')
a.remove('Survived')

In [10]:
print(a)
#print([s.replace('', '') for s in a])
type(a)

['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'FamMeb']


list

In [11]:
import itertools  
all_combinations = []
for r in range(1, len(a) + 1):
    combinations_object = itertools.combinations(a, r)
    combinations_list = combinations_object
    all_combinations += list(combinations_list)
# convert list of tuples to list of lists
res = [list(ele) for ele in all_combinations] 
#print(res)

In [12]:
df = pd.DataFrame()

In [13]:
mod_array = np.array([]) 
mean_array = np.array([]) 
std_array = np.array([]) 
Att_array = np.array([]) 

models = [
    DummyClassifier,
    LogisticRegression,
    DecisionTreeClassifier,
    KNeighborsClassifier,
    GaussianNB,
    SVC,
    RandomForestClassifier,
    xgboost.XGBClassifier,
    #CategoricalNB, #had trouble with this one
]

for b in res:
    #remove all the warnings, trying to update on the model presenting.  Not necessary.
    import warnings
    warnings.simplefilter(action='ignore', category=FutureWarning)
    #define the training model
    X = train_df[b].values
    y = train_df['Survived'].values
    #adjusting X to ensure values are 
    X = preprocessing.StandardScaler().fit(X).transform(X.astype(float))
    #create training and test split.  Test is 30% of data.
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=4)

    #itterating through each of the 7 models and Dummy for reference.
    for m in models:
        cls = m() #not sure why this is needed.  Without it can't get Dummy Classifier to run
        kfold = model_selection.KFold(n_splits=10, random_state=42)
        s = model_selection.cross_val_score(cls, X, y, scoring="roc_auc", cv=kfold)
        #Defining attributes to collect
        mod = f"{m.__name__:22}"
        mean = f"{s.mean():.3f}"
        std = f"{s.std():.2f}"
        Att = str(b)
        
        mod_array = np.append(mod_array, mod)
        mean_array = np.append(mean_array, mean)
        std_array = np.append(std_array, std)
        Att_array = np.append(Att_array, Att)
        
#df = pd.DataFrame([x, m.__name__, s.mean(), s.std(), str(b)])# for b in res)   
        
df = pd.DataFrame(data=[mod_array, mean_array, std_array, Att_array])   
df = df.T
df.columns = ['Model', 'AUC', 'STD', 'Attributes']
df = df.sort_values(by='AUC', ascending=False)
df = df.reset_index()
df = df.drop(['index'], axis=1)
df = df.reset_index()
df['index']=df['index']+1
df = df.rename(columns={"index": "Ranking"})
df.to_csv("Models.csv", index = False)

In [18]:
df.head(20)

,Ranking,Model,AUC,STD,Attributes
0,1,RandomForestClassifier,0.870,0.03,"['Pclass', 'Sex', 'Age', 'Fare', 'FamMeb']"
1,2,RandomForestClassifier,0.868,0.04,"['Pclass', 'Sex', 'Age', 'SibSp', 'Fare', 'Fam..."
2,3,RandomForestClassifier,0.866,0.04,"['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fa..."
3,4,RandomForestClassifier,0.865,0.03,"['Pclass', 'Sex', 'Age', 'SibSp', 'Fare']"
4,5,RandomForestClassifier,0.864,0.03,"['Pclass', 'Sex', 'Age', 'SibSp', 'Fare', 'Emb..."
5,6,RandomForestClassifier,0.861,0.03,"['Pclass', 'Sex', 'Age', 'Fare', 'Embarked', '..."
6,7,RandomForestClassifier,0.861,0.03,"['Pclass', 'Sex', 'Age', 'Fare', 'Embarked']"
7,8,RandomForestClassifier,0.861,0.04,"['Pclass', 'Sex', 'Age', 'Parch', 'Fare', 'Fam..."
8,9,RandomForestClassifier,0.861,0.04,"['Pclass', 'Sex', 'Age', 'SibSp', 'Fare', 'Emb..."
9,10,RandomForestClassifier,0.860,0.04,"['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fa..."


In [15]:
df['AUC'] = df['AUC'].astype(float)
df.dtypes

Ranking         int64
Model          object
AUC           float64
STD            object
Attributes     object
dtype: object

In [16]:
df2 = df.groupby('Model', as_index=False)['Ranking'].mean().sort_values(by='Ranking', ascending=True)

In [17]:
df3 = df.groupby('Model', as_index=False)['AUC'].mean().sort_values(by='AUC', ascending=False)
df2['AUC']=df3['AUC']
df2.head(10)

,Model,Ranking,AUC
7,XGBClassifier,713.529412,0.773184
5,RandomForestClassifier,765.172549,0.767208
6,SVC,848.545098,0.756969
4,LogisticRegression,886.866667,0.747286
3,KNeighborsClassifier,954.705882,0.741973
2,GaussianNB,975.733333,0.738000
0,DecisionTreeClassifier,1109.890196,0.722094
1,DummyClassifier,1909.556863,0.500322


This shows that for this particular dataset, the XGBClassifier has shown the best outcome overall.  The ranking being the average of every test.  For each Model, 255 algorithms are run (2^8 - 1) as 1 would be with no variable tested.  As we have 8 models this is looking at 2040 individual algorithms. 